Gear scrapping calculations for SWGOH

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [74]:
%%writefile bronz.txt
Mk 4 BioTech Implant Prototype
Dark Side Battles: 2-E (Normal)
Dark Side Battles: 4-E (Normal)
Dark Side Battles: 8-H (Normal)
Dark Side Battles: 9-C (Normal)
Light Side Battles: 4-H (Normal)
Light Side Battles: 6-B (Normal)
Light Side Battles: 8-D (Normal)
Light Side Battles: 9-D (Normal)   
Mk 4 BlasTech Weapon Mod Prototype
Dark Side Battles: 2-H (Normal)
Dark Side Battles: 5-J (Normal)
Dark Side Battles: 7-G (Normal)
Dark Side Battles: 9-A (Normal)
Dark Side Battles: 9-B (Normal)
Light Side Battles: 4-E (Normal)
Mk 4 Loronar Power Cell Prototype
Dark Side Battles: 2-C (Normal)
Dark Side Battles: 5-F (Normal)
Dark Side Battles: 6-J (Normal)
Dark Side Battles: 8-B (Normal)
Dark Side Battles: 9-D (Normal)
Fleet Side Battles: 2-A (Normal)
Mk 5 Fabritech Data Pad
Light Side Battles: 5-K (Normal)
Light Side Battles: 6-C (Normal)
Light Side Battles: 7-B (Normal)
Light Side Battles: 8-A (Normal)
Light Side Battles: 9-B (Normal)
Fleet Side Battles: 1-D (Normal)

Overwriting bronz.txt


In [169]:
with open('bronz.txt', 'r+') as f:
    with open('bronz_new.csv', 'w+') as outf:
        f.seek(0)
        outf.truncate()
        #1st row of csv
        outf.write('gear_name;DS;LS;FL\n')
        
        #data for 'item' column
        items = []
        for line in f.readlines():
            if line.startswith('Mk'):
                items.append(line[:-1])
        
        #list of separate data for each item
        f.seek(0)
        by_items = f.read().split('Mk')[1:]
        all_darks_per_item = []
        all_lights_per_item = []
        all_fleets_per_item = []
        for item in by_items:
            item = item.strip().split('\n')[1:]         
            dark = []
            light = []
            fleet = []
            
            #collect darks per item
            for line in item:  
                if 'Dark' in line: 
                    dark.append(line.split()[3])
            all_darks_per_item.append(dark)
            
            #collect lights per item
            for line in item:  
                if 'Light' in line: 
                    light.append(line.split()[3])
            all_lights_per_item.append(light)
            
            #collect fleets per item
            for line in item:  
                if 'Fleet' in line: 
                    fleet.append(line.split()[3])
            all_fleets_per_item.append(fleet)
        
        #write to csv: name_of_item;list_of_darks;list_of_lights;list_of_leets
        for i in range(0,len(items)):
            outf.write(f'{items[i]};{all_darks_per_item[i]};{all_lights_per_item[i]};{all_fleets_per_item[i]}\n')
        

In [170]:
with open('bronz_new.csv', 'r') as f:
    print(f.read())

gear_name;DS;LS;FL
Mk 4 BioTech Implant Prototype;['2-E', '4-E', '8-H', '9-C'];['4-H', '6-B', '8-D', '9-D'];[]
Mk 4 BlasTech Weapon Mod Prototype;['2-H', '5-J', '7-G', '9-A', '9-B'];['4-E'];[]
Mk 4 Loronar Power Cell Prototype;['2-C', '5-F', '6-J', '8-B', '9-D'];[];['2-A']
Mk 5 Fabritech Data Pad;[];['5-K', '6-C', '7-B', '8-A', '9-B'];['1-D']



In [186]:
bronz = pd.read_csv('bronz_new.csv', sep=';')

In [172]:
def to_1D(series):
 return pd.Series([x for _list in series for x in _list])

In [187]:
bronz['DS'] = bronz['DS'].apply(eval)
bronz['LS'] = bronz['LS'].apply(eval)
bronz['FL'] = bronz['FL'].apply(eval)

In [188]:
bronz

,gear_name,DS,LS,FL
0,Mk 4 BioTech Implant Prototype,"[2-E, 4-E, 8-H, 9-C]","[4-H, 6-B, 8-D, 9-D]",[]
1,Mk 4 BlasTech Weapon Mod Prototype,"[2-H, 5-J, 7-G, 9-A, 9-B]",[4-E],[]
2,Mk 4 Loronar Power Cell Prototype,"[2-C, 5-F, 6-J, 8-B, 9-D]",[],[2-A]
3,Mk 5 Fabritech Data Pad,[],"[5-K, 6-C, 7-B, 8-A, 9-B]",[1-D]


In [189]:
bronz_explode = bronz.explode('DS').explode('LS').explode('FL')

In [195]:
bronz_explode.head()

,gear_name,DS,LS,FL
0,Mk 4 BioTech Implant Prototype,2-E,4-H,NaN
0,Mk 4 BioTech Implant Prototype,2-E,6-B,NaN
0,Mk 4 BioTech Implant Prototype,2-E,8-D,NaN
0,Mk 4 BioTech Implant Prototype,2-E,9-D,NaN
0,Mk 4 BioTech Implant Prototype,4-E,4-H,NaN


In [196]:
bronz['all'] = bronz['DS']+bronz['LS']+bronz['FL']

In [198]:
to_1D(bronz['all']).value_counts()

9-D    2
9-B    2
4-E    2
8-B    1
8-H    1
4-H    1
8-A    1
9-A    1
6-C    1
7-G    1
2-A    1
5-F    1
1-D    1
7-B    1
8-D    1
2-C    1
9-C    1
6-B    1
5-J    1
6-J    1
5-K    1
2-E    1
2-H    1
dtype: int64

In [202]:
%%writefile chrom.txt
Mk 1 Czerka Stun Cuffs
Dark Side Battles: 3-A (Normal)
Dark Side Battles: 5-D (Normal)
Dark Side Battles: 7-H (Normal)
Light Side Battles: 2-G (Normal)
Mk 2 Sienar Holo Projector Prototype Salvage
Dark Side Battles: 3-H (Normal)
Dark Side Battles: 7-A (Normal)
Dark Side Battles: 9-D (Normal)
Light Side Battles: 4-A (Normal)
Light Side Battles: 6-E (Normal)
Light Side Battles: 9-F (Normal)
Fleet Side Battles: 1-C (Normal)
Fleet Side Battles: 2-D (Normal)
Fleet Side Battles: 4-E (Normal)
Fleet Side Battles: 4-F (Normal)
Mk 3 BlasTech Weapon Mod
Light Side Battles: 3-C (Normal)
Light Side Battles: 5-G (Normal)
Light Side Battles: 7-H (Normal)
Light Side Battles: 8-F (Normal)

Overwriting chrom.txt


In [203]:
with open('chrom.txt', 'r+') as f:
    with open('chrom_new.csv', 'w+') as outf:
        f.seek(0)
        outf.truncate()
        #1st row of csv
        outf.write('gear_name;DS;LS;FL\n')
        
        #data for 'item' column
        items = []
        for line in f.readlines():
            if line.startswith('Mk'):
                items.append(line[:-1])
        
        #list of separate data for each item
        f.seek(0)
        by_items = f.read().split('Mk')[1:]
        all_darks_per_item = []
        all_lights_per_item = []
        all_fleets_per_item = []
        for item in by_items:
            item = item.strip().split('\n')[1:]         
            dark = []
            light = []
            fleet = []
            
            #collect darks per item
            for line in item:  
                if 'Dark' in line: 
                    dark.append(line.split()[3])
            all_darks_per_item.append(dark)
            
            #collect lights per item
            for line in item:  
                if 'Light' in line: 
                    light.append(line.split()[3])
            all_lights_per_item.append(light)
            
            #collect fleets per item
            for line in item:  
                if 'Fleet' in line: 
                    fleet.append(line.split()[3])
            all_fleets_per_item.append(fleet)
        
        #write to csv: name_of_item;list_of_darks;list_of_lights;list_of_leets
        for i in range(0,len(items)):
            outf.write(f'{items[i]};{all_darks_per_item[i]};{all_lights_per_item[i]};{all_fleets_per_item[i]}\n')

In [204]:
with open('chrom_new.csv', 'r') as f:
    print(f.read())

gear_name;DS;LS;FL
Mk 1 Czerka Stun Cuffs;['3-A', '5-D', '7-H'];['2-G'];[]
Mk 2 Sienar Holo Projector Prototype Salvage;['3-H', '7-A', '9-D'];['4-A', '6-E', '9-F'];['1-C', '2-D', '4-E', '4-F']
Mk 3 BlasTech Weapon Mod;[];['3-C', '5-G', '7-H', '8-F'];[]



In [205]:
chrom = pd.read_csv('chrom_new.csv', sep=';')

In [207]:
chrom['DS'] = chrom['DS'].apply(eval)
chrom['LS'] = chrom['LS'].apply(eval)
chrom['FL'] = chrom['FL'].apply(eval)

In [208]:
chrom

,gear_name,DS,LS,FL
0,Mk 1 Czerka Stun Cuffs,"[3-A, 5-D, 7-H]",[2-G],[]
1,Mk 2 Sienar Holo Projector Prototype Salvage,"[3-H, 7-A, 9-D]","[4-A, 6-E, 9-F]","[1-C, 2-D, 4-E, 4-F]"
2,Mk 3 BlasTech Weapon Mod,[],"[3-C, 5-G, 7-H, 8-F]",[]


In [209]:
chrom_explode = chrom.explode('DS').explode('LS').explode('FL')

In [210]:
chrom['all'] = chrom['DS']+chrom['LS']+chrom['FL']

In [212]:
to_1D(chrom['all']).value_counts()

7-H    2
2-G    1
8-F    1
9-F    1
5-G    1
4-F    1
4-A    1
6-E    1
4-E    1
5-D    1
9-D    1
7-A    1
3-C    1
2-D    1
3-H    1
3-A    1
1-C    1
dtype: int64

In [214]:
bronz_chrom = pd.concat([bronz, chrom])

In [215]:
bronz_chrom

,gear_name,DS,LS,FL,all
0,Mk 4 BioTech Implant Prototype,"[2-E, 4-E, 8-H, 9-C]","[4-H, 6-B, 8-D, 9-D]",[],"[2-E, 4-E, 8-H, 9-C, 4-H, 6-B, 8-D, 9-D]"
1,Mk 4 BlasTech Weapon Mod Prototype,"[2-H, 5-J, 7-G, 9-A, 9-B]",[4-E],[],"[2-H, 5-J, 7-G, 9-A, 9-B, 4-E]"
2,Mk 4 Loronar Power Cell Prototype,"[2-C, 5-F, 6-J, 8-B, 9-D]",[],[2-A],"[2-C, 5-F, 6-J, 8-B, 9-D, 2-A]"
3,Mk 5 Fabritech Data Pad,[],"[5-K, 6-C, 7-B, 8-A, 9-B]",[1-D],"[5-K, 6-C, 7-B, 8-A, 9-B, 1-D]"
0,Mk 1 Czerka Stun Cuffs,"[3-A, 5-D, 7-H]",[2-G],[],"[3-A, 5-D, 7-H, 2-G]"
1,Mk 2 Sienar Holo Projector Prototype Salvage,"[3-H, 7-A, 9-D]","[4-A, 6-E, 9-F]","[1-C, 2-D, 4-E, 4-F]","[3-H, 7-A, 9-D, 4-A, 6-E, 9-F, 1-C, 2-D, 4-E, ..."
2,Mk 3 BlasTech Weapon Mod,[],"[3-C, 5-G, 7-H, 8-F]",[],"[3-C, 5-G, 7-H, 8-F]"


In [219]:
to_1D(bronz_chrom['FL']).value_counts()

2-D    1
1-D    1
4-F    1
2-A    1
1-C    1
4-E    1
dtype: int64

Pointless... I forgot values in columns are indistinguishable once merged into 'all'. 5h wasted doing pointless transformations.

30 min break, and I figured out how to do it in a waaaay more pandas way. Waaay easier.

In [262]:
 def gear_to_array(txt_file):
    with open(txt_file, 'r+') as f:
        out = []
        name = ''
        for line in f.readlines():
            if 'Mk' in line:
                name = line[:-1]
                continue
            line = line.split()
            out.extend((name, line[0], line[3]))
        return np.reshape(out, (len(out)//3, 3))
    #new_out1 = [out[i:i+3] for i in range(0, len(out), 3)]

In [263]:
chrom_data = gear_to_array('chrom.txt')

In [276]:
df_chrom = pd.DataFrame(chrom_data, columns=['gear_name','battle_type', 'battle_location'])
df_chrom.head()

,gear_name,battle_type,battle_location
0,Mk 1 Czerka Stun Cuffs,Dark,3-A
1,Mk 1 Czerka Stun Cuffs,Dark,5-D
2,Mk 1 Czerka Stun Cuffs,Dark,7-H
3,Mk 1 Czerka Stun Cuffs,Light,2-G
4,Mk 2 Sienar Holo Projector Prototype Salvage,Dark,3-H


In [265]:
df_chrom[df_chrom['battle_type']=='Dark']['battle_location'].value_counts()

3-H    1
9-D    1
7-A    1
7-H    1
3-A    1
5-D    1
Name: battle_location, dtype: int64

In [267]:
bronz_data = gear_to_array('bronz.txt')

In [275]:
df_bronz = pd.DataFrame(bronz_data, columns=['gear_name','battle_type', 'battle_location'])
df_bronz.head()

,gear_name,battle_type,battle_location
0,Mk 4 BioTech Implant Prototype,Dark,2-E
1,Mk 4 BioTech Implant Prototype,Dark,4-E
2,Mk 4 BioTech Implant Prototype,Dark,8-H
3,Mk 4 BioTech Implant Prototype,Dark,9-C
4,Mk 4 BioTech Implant Prototype,Light,4-H


In [274]:
df_bronz_chrom = pd.concat([df_bronz, df_chrom])
df_bronz_chrom.head()

,gear_name,battle_type,battle_location
0,Mk 4 BioTech Implant Prototype,Dark,2-E
1,Mk 4 BioTech Implant Prototype,Dark,4-E
2,Mk 4 BioTech Implant Prototype,Dark,8-H
3,Mk 4 BioTech Implant Prototype,Dark,9-C
4,Mk 4 BioTech Implant Prototype,Light,4-H


In [273]:
df_bronz_chrom[df_bronz_chrom['battle_type'] == 'Dark']['battle_location'].value_counts()

9-D    2
8-B    1
4-E    1
3-A    1
9-A    1
7-A    1
7-G    1
5-D    1
5-F    1
8-H    1
7-H    1
2-C    1
9-C    1
3-H    1
5-J    1
6-J    1
9-B    1
2-E    1
2-H    1
Name: battle_location, dtype: int64